# Visualising Eye Movement

In [33]:
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import animation
import seaborn as sns
import numpy as np

import matplotlib
matplotlib.use("Agg") 

In [34]:
import sys
import os

sys.path.append(os.path.abspath("../src"))  # Add folder_a to sys.path
from config import *

In [35]:
experiment = "EVIL_BASTARD"
participant_ids = pd.read_parquet(
        f"{PREPROCESSED_DIR}/{experiment}_samples.pq", 
        columns=["participant_id"]
    )

participant_ids = participant_ids["participant_id"].unique()

In [36]:
# Read data for participant
participant_id = participant_ids[10]
filters = [('participant_id', '=', str(participant_id))]

samples_df = pd.read_parquet(
    f"{PREPROCESSED_DIR}/{experiment}_samples.pq",
    filters=[('participant_id', '=', participant_id)]
    )

events_df = pd.read_parquet(
    f"{PREPROCESSED_DIR}/{experiment}_events.pq",
    filters=[('participant_id', '=', participant_id)]
    )

In [37]:
events_df

,experiment,participant_id,trial_id,time,event,eye,start_time,end_time,duration,x,...,end_y,amplitude,peak_velocity,angle,speed,target_x,target_y,colour,stimulus_x,stimulus_y
0,EVIL_BASTARD,146,0,0,START,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
1,EVIL_BASTARD,146,0,1,ESACC,R,1.0,72.0,71.0,NaN,...,540.7,NaN,649.0,NaN,NaN,NaN,NaN,None,NaN,NaN
2,EVIL_BASTARD,146,0,66,EFIX,L,66.0,289.0,223.0,630.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
3,EVIL_BASTARD,146,0,73,EFIX,R,73.0,290.0,217.0,609.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
4,EVIL_BASTARD,146,0,290,ESACC,L,290.0,319.0,29.0,NaN,...,541.6,5.31,387.0,NaN,NaN,NaN,NaN,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52265,EVIL_BASTARD,146,23,11669,FIXPOINT,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,255 0 0,-5.0,274.0
52266,EVIL_BASTARD,146,23,11673,FIXPOINT,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,255 0 0,-6.0,274.0
52267,EVIL_BASTARD,146,23,11678,FIXPOINT,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,255 0 0,-6.0,274.0
52268,EVIL_BASTARD,146,23,11678,TRIAL_VAR_DATA,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.5,NaN,NaN,255 0 0,-6.0,274.0


In [ ]:
trial_id=0

sample_df = samples_df.loc[samples_df["trial_id"]==trial_id,:]

# Extract fixpoints
event_df = events_df.loc[events_df["trial_id"]==trial_id,:]
fixpoints_df = event_df[event_df["event"]=="FIXPOINT"].loc[:,["participant_id", "trial_id", "time", "event", "colour", "stimulus_x", "stimulus_y"]]

# Insert fixpoints in sample data
# Ensure you are modifying actual copies
sample_df = sample_df.copy()
fixpoints_df = fixpoints_df.copy()

# Sort by participant_id, trial_id, then time (important for merge_asof with 'by')
sample_df = sample_df.sort_values(["time"])
fixpoints_df = fixpoints_df.sort_values(["time"])

# Rename 'colour' column to 'fixpoint' so it's ready to merge
fixpoints_df = fixpoints_df.rename(columns={"colour": "fixpoint"})

# Perform a backward-looking join: for each row in sample_df, find the most recent fixpoint time
sample_df = pd.merge_asof(
    sample_df,
    fixpoints_df,
    on=["time"],
    by=["participant_id", "trial_id"],
    direction="nearest",
    tolerance=10
)

sample_df["fixpoint"] = sample_df["fixpoint"].map({RED:"red", GREEN:"green", BLUE:"blue", WHITE:"white"})

In [39]:
x = sample_df.copy()
x_sampled = x[x.index % 100 == 0]

# Apply Seaborn style
sns.set(style="whitegrid")

fig, ax = plt.subplots(1, 1, figsize = (6, 6))

# Initialize the scatter plot object
scatter_right = ax.scatter([], [], cmap='viridis', label="Right", alpha=0.6)  # 's' is for size, 'c' is for color
scatter_left = ax.scatter([], [], cmap='coolwarm', label="Left", alpha=0.6)  # 's' is for size, 'c' is for color
scatter_fixpoint = ax.scatter([], [], label="Fixpoint", alpha=0.6, s=50)  # 's' is for size, 'c' is for color

x_min, x_max = 0, 1920 
y_min, y_max = 0, 1080 
ax.set_xlim([x_min, x_max]) 
ax.set_ylim([y_min, y_max]) 

# Add legend to distinguish between the three points
ax.legend()

def animate(i):
    # Update the scatter plot data for each frame
    
    # Update coordinates
    coords_right = np.c_[x_sampled["x_left"].iloc[i], x_sampled["y_right"].iloc[i]]
    coords_left = np.c_[x_sampled["x_left"].iloc[i], x_sampled["y_left"].iloc[i]] 
    coords_fixpoint = np.c_[x_sampled["stimulus_x"].iloc[i], x_sampled["stimulus_y"].iloc[i]] 

    scatter_right.set_offsets(coords_right)    
    scatter_left.set_offsets(coords_left)
    scatter_fixpoint.set_offsets(coords_fixpoint)

    # Update colour based on fixpoint
    fixpoint_color = x_sampled["fixpoint"].iloc[i]
    if pd.isna(fixpoint_color):
        fixpoint_color = "gray"
    elif fixpoint_color == "white":
        fixpoint_color = "black"
    scatter_fixpoint.set_color([fixpoint_color])

    return scatter_right, scatter_left, scatter_fixpoint

anim = animation.FuncAnimation(
    fig, 
    animate, 
    frames = len(x_sampled), 
    interval=100,
    blit = False
)

anim.save(f"Animations/{experiment}_{participant_id}_{trial_id}.mp4", writer="ffmpeg", fps=30)



/tmp/ipykernel_20230/2851728046.py:10: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  scatter_right = ax.scatter([], [], cmap='viridis', label="Right", alpha=0.6)  # 's' is for size, 'c' is for color
/tmp/ipykernel_20230/2851728046.py:11: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  scatter_left = ax.scatter([], [], cmap='coolwarm', label="Left", alpha=0.6)  # 's' is for size, 'c' is for color
2025-04-21 11:27:27,072 - INFO - animation.save:1076 - Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
2025-04-21 11:27:27,074 - INFO - animation._run:319 - MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 600x600 -pix_fmt rgba -framerate 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -y Animations/EVIL_BASTARD_146_0.mp4
